# V - Automatic Model Selection from Parametric Sweep using Task Dependencies
In this notebook we will be taking the example from the [Parametric Sweep](04_Parameter_Sweep.ipynb) notebook and automating the entire chain using task dependencies in a single Azure Batch job.

* [Setup](#section1)
* [Configure job](#section2)
* [Submit job](#section3)
* [Download best model](#section4)
* [Delete job](#section5)

<a id='section1'></a>

## Setup

Create a simple alias for Batch Shipyard

In [ ]:
%alias shipyard SHIPYARD_CONFIGDIR=config python $HOME/batch-shipyard/shipyard.py %l

Check that everything is working

In [ ]:
shipyard

Read in the account information we saved earlier

In [ ]:
import json

def read_json(filename):
    with open(filename, 'r') as infile:
        return json.load(infile)
    
account_info = read_json('account_information.json')

storage_account_key = account_info['storage_account_key']
storage_account_name = account_info['storage_account_name']
IMAGE_NAME = account_info['IMAGE_NAME']
STORAGE_ALIAS = account_info['STORAGE_ALIAS']

<a id='section2'></a>

## Configure Job

As in the previous job we ran on a single node we will be running the job on GPU enabled nodes. We will be repeating the parametric search we did in the previous notebook but this time instead of downloading all the results and evaluating the model performance we will get a final task to do that for us using task dependencies.

In [ ]:
import os

def write_json_to_file(json_dict, filename):
    """ Simple function to write JSON dictionaries to files
    """
    with open(filename, 'w') as outfile:
        json.dump(json_dict, outfile)


Generate the `jobs.json` configuration file

In [ ]:
JOB_ID = 'cntk-ps-as-job'

jobs = {
    "job_specifications": [
        {
            "id": JOB_ID,
            "tasks": [
                        {
                            "image": IMAGE_NAME,
                            "task_factory": {
                                "parametric_sweep": {
                                    "product": [
                                        {#num_convolution_layers
                                            "start": 2,
                                            "stop": 4,
                                            "step": 1
                                        },
                                        {#minibatch_size
                                            "start": 32,
                                            "stop": 96,
                                            "step": 32
                                        }
                                    ]
                                }
                            },
                            "command": "bash -c \"source /cntk/activate-cntk; python -u ConvNet_CIFAR10.py --datadir $AZ_BATCH_NODE_SHARED_DIR/data --num_convolution_layers {0} --minibatch_size {1} --max_epochs 30\"",
                            "remove_container_after_exit": True,
                            "gpu": True,
                            "resource_files": [
                                    {
                                        "file_path": "ConvNet_CIFAR10.py",
                                        "blob_source": "https://batchshipyardexamples.blob.core.windows.net/code/ConvNet_CIFAR10.py",
                                        "file_mode":'0777'
                                    }
                            ],
                            "output_data": {
                                "azure_storage": [
                                    {
                                        "storage_account_settings": STORAGE_ALIAS,
                                        "container": "output",
                                        "source": "$AZ_BATCH_TASK_DIR/wd/Models"
                                    },
                                ]
                            },
                        }
            ]  
        }
    ]
}

num_parameter_sweep_tasks = 4
print('number of tasks for parametric sweep {}: {}'.format(JOB_ID, num_parameter_sweep_tasks))

Now we'll create the Python program to run that performs the best model selection. Note that this code is nearly similar to the code for selecting the best model locally in the [Parameter sweep notebook](04_Parameter_Sweep.ipynb).

In [ ]:
%%writefile autoselect.py
import json
import os
import shutil

def read_json(filename):
    with open(filename, 'r') as infile:
        return json.load(infile)

def scandir(basedir):
    for root, dirs, files in os.walk(basedir):
        for f in files:
            yield os.path.join(root, f) 

MODELS_DIR = os.path.join('wd', 'Models')
            
results_dict = {}
for model in scandir(MODELS_DIR):
    if not model.endswith('.json'):
        continue
    key = model.split(os.sep)[2]  # due to MODELS_DIR path change
    results_dict[key] = read_json(model)

# use items() instead of iteritems() as this will be run in python3
tuple_min_error = min(results_dict.items(), key=lambda x: x[1]['test_metric'])
configuration_with_min_error = tuple_min_error[0]
print('task with smallest error: {} ({})'.format(configuration_with_min_error, tuple_min_error[1]['test_metric']))

# copy best model to wd
MODEL_NAME = 'ConvNet_CIFAR10_model.dnn'
shutil.copy(os.path.join(MODELS_DIR, configuration_with_min_error, MODEL_NAME), '.')

We now need to prepare the file to be uploaded to the Azure Storage account to be referenced in the task:

In [ ]:
INPUT_CONTAINER = 'input-autoselect'
OUTPUT_CONTAINER = 'output-autoselect'
UPLOAD_DIR = 'autoselect_upload'

!rm -rf $UPLOAD_DIR
!mkdir -p $UPLOAD_DIR
!mv autoselect.py $UPLOAD_DIR
!ls -alF $UPLOAD_DIR

Alias `blobxfer` and upload it to `INPUT_CONTAINER`:

In [ ]:
%alias blobxfer python -m blobxfer

In [ ]:
blobxfer $storage_account_name $INPUT_CONTAINER $UPLOAD_DIR --upload --storageaccountkey $storage_account_key

Now we'll append the `auto-model-selection` task which depends on the prior training tasks. The important properties here are `depends_on_range` which specifies a range of task ids the `auto-model-selection` task depends on. Additionally, this task requires data from the prior run task which is specified in `input_data`.

In [ ]:
def generate_input_data_spec(job_id, task_id):
    return {
        "job_id": job_id,
        "task_id": task_id,
        "include": ["wd/Models/*_{}_{}/*".format(task_id, job_id)]
    }

input_data = []
for x in range(0, num_parameter_sweep_tasks):
    input_data.append(generate_input_data_spec(JOB_ID, '{}'.format(x)))

model_selection_task = {
    "id": "auto-model-selection",
    "command": 'bash -c "source /cntk/activate-cntk; python -u autoselect.py"',
    "depends_on_range": [0, num_parameter_sweep_tasks - 1],
    "image": IMAGE_NAME,
    "remove_container_after_exit": True,
    "input_data": {
        "azure_batch": input_data,
        "azure_storage": [
            {
                "storage_account_settings": STORAGE_ALIAS,
                "container": INPUT_CONTAINER
            }
        ]
    },
    "output_data": {
        "azure_storage": [
            {
                "storage_account_settings": STORAGE_ALIAS,
                "container": OUTPUT_CONTAINER,
                "include": ["*wd/ConvNet_CIFAR10_model.dnn"],
                "blobxfer_extra_options": "--delete --strip-components 2"
            }
        ]
    }
}

# append auto-model-selection task to jobs
jobs['job_specifications'][0]['tasks'].append(model_selection_task)

In [ ]:
write_json_to_file(jobs, os.path.join('config', 'jobs.json'))
print(json.dumps(jobs, indent=4, sort_keys=True))

<a id='section3'></a>

## Submit job
Check that everything is ok with our pool before we submit our jobs

In [ ]:
shipyard pool list

Now that we have confirmed everything is working we can execute our job using the command below. 

In [ ]:
shipyard jobs add

Using the command below we can check the status of our jobs. Once all jobs have an exit code we can continue. You can also view the **heatmap** of this pool on [Azure Portal](https://portal.azure.com) to monitor the progress of this job on the compute nodes under your Batch account.

In [ ]:
shipyard jobs listtasks --jobid $JOB_ID

<a id='section4'></a>

## Download best model
The best performing model from the parametric sweep job should now be saved to our `OUTPUT_CONTAINER` container by the `auto-model-selection` task. Let's save this model in `MODELS_DIR`:

In [ ]:
MODELS_DIR = 'auto-selected-model'

Download the best performing model:

In [ ]:
blobxfer $storage_account_name $OUTPUT_CONTAINER $MODELS_DIR --remoteresource . --download --storageaccountkey $storage_account_key

The best model file (`ConvNet_CIFAR10_model.dnn`) is now ready for use.

In [ ]:
!ls -alF $MODELS_DIR

<a id='section5'></a>

## Delete job

To delete the job use the command below. Just be aware that this will get rid of all the files created by the job and tasks.

In [ ]:
shipyard jobs del -y --termtasks --wait